### Load_Census_Data
#### This script imports 2017 Peruvian national census data and distribute to 1km x 1km grids. The original census data was obtained from the [Nathional Institute of Statistics and Information (INEI)](http://censos2017.inei.gob.pe/redatam/).


Donghoon Lee, Nov-3-2019

In [1]:
import os
import numpy as np
import pandas as pd
import gdal
import rasterio
import fhvuln as fhv

### Load 2017 INEI Census data:
- C2P1: Type of Housing (hous, 0-1)
- PAGE5: Percent children under 5 years
- PAGE65: Percent of elderly population (65+ years)
- PFEMALE: Percent females
- PDISABL: Percent population with disability
- PMEDINS: Percent population with health insurance
- PNOSWALL: Percent households without strong walls
- PNOWATER: Percent households without public water supply
- PNOELEC: Percent households without electricity
- PNOSEWAGE: Percent households without sewage infrastructure
- PLITERACY: Percent population who cannot read and write
- PNOPRIEDU: Percent population who don't complete primary education
- PNOCOLLEGE: Percent population who don't complete college degree
- PRENT: Percentage of rented houses
- ANUMBFAMILY: Averaged Numer of people in family
- PPHONE: Percent households with cell phone or landline
- PAUTOMOBILE: Percent households with automobiles

In [2]:
input_names = [['PAGE5','pos','person','Children under 5 years (%)'],
               ['PAGE65','pos','person','Elder population (65+ years) (%)'],
               ['PFEMALE','pos','person','Percent females'],
               ['PDISABL','pos','person','Percent population with disability'],
               ['PMEDINS','neg','person','Percent population with health insurance'],
               ['PNOSWALL','pos','house','Percent households without strong walls'],
               ['PNOWATER','pos','house','Percent households without public water supply'],
               ['PNOELEC','pos','house','Percent households without electricity'],
               ['PNOSEWAGE','pos','house','Percent households without sewage infrastructure'],
               ['PLITERACY','pos','person','Percent population who cannot read and write'],
               ['PNOPRIEDU','pos','person','Percent population who don''t complete primary education'],
               ['PNOCOLLEGE','pos','person','Percent population who don''t complete college degree'],
               ['PRENT','pos','house','Percentage of rented houses'],
               ['ANUMBFAMILY','pos','house','Averaged Numer of people in family'],
               ['PPHONE','neg','house','Percent households with cell phone or landline'],
               ['PAUTOMOBILE','neg','house','Percent households with automobiles']]
data_name = pd.DataFrame(input_names, columns=['Name','Sign','Type','Description'])

In [3]:
# Label values is manually exported and translated
fn_label = os.path.join('data','census','label_values.xlsx')

# C2P1: Type of Housing (hous, 0-1) 
# (#house_Kutcha_and_Jhupri / #house_total)
# *Pucca means high quality materials (e.g., cement or RCC)
# *Kutcha & Jhupri means weaker materials (e.g., mud, clay, lime, or thatched)
df1 = fhv.ineiCensus(os.path.join('data','census','C2P1.xlsx'), fn_label)
totalHous1 = df1.sum(axis=1).sum()
df2 = fhv.ineiCensus(os.path.join('data','census','C2P2.xlsx'), fn_label)
totalHous2 = df2.sum(axis=1).sum()


# POPULATION DATA
"""
Surveyed population:  29,381,884
Ommited population:    1,855,501
Total population:     31,237,385  
"""
df = fhv.ineiCensus(os.path.join('data','census','EDQUINQ.xlsx'), fn_label)
df = fhv.distCorrect(df, 'sum')
data = pd.DataFrame(index=df.index)  # This should be located at here (#IDDIST=1873)
popu = df.sum(axis=1)           # Total population: 29381884
# - PAGE5 Percent children under 5 years
data['PAGE5'] = df[df.columns[1]]/df.sum(axis=1)
# - PAGE65: Percent of elderly population (65+ years)
data['PAGE65'] = df[df.columns[14:]].sum(axis=1)/df.sum(axis=1)
# - PFEMALE: Percent females
df = fhv.ineiCensus(os.path.join('data','census','C5P2.xlsx'), fn_label)
df = fhv.distCorrect(df, 'sum')
data['PFEMALE'] = df['Woman']/df.sum(axis=1)

#########
# Percent housholds that are female owned
# *** DOWNLOAD ***
#########

# DISABILITY
# - PDISABL: Percent population with disability
df = fhv.ineiCensus(os.path.join('data','census','P09DISC.xlsx'), fn_label)
df = fhv.distCorrect(df, 'sum')
data['PDISABL'] = 1 - df[df.columns[-1]]/df.sum(axis=1)

# MEDICAL INSURANCE
# - PMEDINS: Percent population with health insurance
df = fhv.ineiCensus(os.path.join('data','census','P08AFILIA.xlsx'), fn_label)
df = fhv.distCorrect(df, 'sum')
data['PMEDINS']= 1 - df[df.columns[-1]]/df.sum(axis=1)

# BUILT ENVIRONMENT
# - PNOSWALL: Percent households without strong walls
df = fhv.ineiCensus(os.path.join('data','census','C2P3.xlsx'), fn_label)
df = fhv.distCorrect(df, 'sum')
hous = df.sum(axis=1)           # Total households 7698900
data['PNOSWALL'] = 1 - df[df.columns[1:4]].sum(axis=1)/df.sum(axis=1)
# - PNOWATER: Percent household without public water supply
df = fhv.ineiCensus(os.path.join('data','census','C2P6.xlsx'), fn_label)
df = fhv.distCorrect(df, 'sum')
data['PNOWATER'] = 1 - df[df.columns[1:3]].sum(axis=1)/df.sum(axis=1)
# - PNOELEC: Percent household without electricity
df = fhv.ineiCensus(os.path.join('data','census','C2P11.xlsx'), fn_label)
df = fhv.distCorrect(df, 'sum')
data['PNOELEC'] = df[df.columns[2]]/df.sum(axis=1)
# - PNOSEWAGE: Percent household without sewage infrastructure
# This excludes 'Public drainage network within the dwelling' and
# 'Public drainage network outside the home, but inside the building'
df = fhv.ineiCensus(os.path.join('data','census','C2P10.xlsx'), fn_label)
df = fhv.distCorrect(df, 'sum')
data['PNOSEWAGE']= 1 - df[df.columns[1:3]].sum(axis=1)/df.sum(axis=1)

# EDUCATION
# - PLITERACY: Percent population who cannot read and write
df = fhv.ineiCensus(os.path.join('data','census','C5P12.xlsx'), fn_label)
df = fhv.distCorrect(df, 'sum')
data['PLITERACY'] = df[df.columns[2]]/df.sum(axis=1)
# - PNOPRIEDU: Percent population who don't complete primary education
df = fhv.ineiCensus(os.path.join('data','census','C5P13NIV.xlsx'), fn_label)
df = fhv.distCorrect(df, 'sum')
data['PNOPRIEDU'] = df[df.columns[1:3]].sum(axis=1)/df.sum(axis=1)
# - PNOCOLLEGE: Percent population who don't complete college degree
data['PNOCOLLEGE'] = df[df.columns[1:-2]].sum(axis=1)/df.sum(axis=1)

# HOUSING
# - PRENT: Percentage of rented houses
# Includes: 'Rented', 'Assignment', 'Another way'
df = fhv.ineiCensus(os.path.join('data','census','C2P13.xlsx'), fn_label)
df = fhv.distCorrect(df, 'sum')
data['PRENT'] = df[df.columns[[1,4,5]]].sum(axis=1)/df.sum(axis=1)

# FAMILY STRUCTURE
# - ANUMBFAMILY: Averaged Numer of people in family
df = fhv.ineiCensus(os.path.join('data','census','C4P1.xlsx'), fn_label)
df = fhv.distCorrect(df, 'sum')
data['ANUMBFAMILY'] = df[df.columns[1:]].values.dot(np.arange(31))/df.sum(axis=1)

# SOCIOECONOMIC CHARACTERISTICS
########
#UPDATE: Find cross table from REDATUM
########
# - PPHONE: Percent households with cell phone or landline
df = fhv.ineiCensus(os.path.join('data','census','C3P210.xlsx'), fn_label)
df = fhv.distCorrect(df, 'sum')
data['PPHONE'] = df[df.columns[1]]/df.sum(axis=1)
# - PAUTOMOBILE: Percent households with automobiles
df = fhv.ineiCensus(os.path.join('data','census','C3P214.xlsx'), fn_label)
df = fhv.distCorrect(df, 'sum')
data['PAUTOMOBILE'] = df[df.columns[1]]/df.sum(axis=1)

/Users/dlee/anaconda3/envs/dlee_sovi/lib/python3.6/site-packages/pandas/core/generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [4]:
print(data.shape)
data

(1873, 16)


,PAGE5,PAGE65,PFEMALE,PDISABL,PMEDINS,PNOSWALL,PNOWATER,PNOELEC,PNOSEWAGE,PLITERACY,PNOPRIEDU,PNOCOLLEGE,PRENT,ANUMBFAMILY,PPHONE,PAUTOMOBILE
IDDIST,,,,,,,,,,,,,,,,
10101,0.080794,0.081132,0.526650,0.099573,0.818497,0.049218,0.096125,0.075867,0.146295,0.089122,0.090089,0.848434,0.278042,3.652009,0.926905,0.106472
10102,0.083969,0.179389,0.488550,0.251908,0.954198,0.720000,0.010000,0.150000,0.220000,0.184000,0.156000,0.960000,0.190000,2.466667,0.625000,0.000000
10103,0.097711,0.083627,0.475352,0.090669,0.817782,0.585586,0.330330,0.276276,0.666667,0.178236,0.169794,0.981238,0.201201,3.000000,0.695775,0.036620
10104,0.068536,0.115265,0.506231,0.182243,0.889408,0.278302,0.122642,0.122642,0.169811,0.099673,0.099673,0.968954,0.132075,2.918182,0.740909,0.022727
10105,0.083761,0.140171,0.482051,0.073504,0.947009,0.446078,0.200980,0.343137,0.583333,0.186715,0.179533,0.987433,0.102941,2.839806,0.567961,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250303,0.121860,0.034836,0.462704,0.089614,0.645947,0.915217,0.696739,0.235326,0.915761,0.146763,0.142578,0.985631,0.137500,3.998942,0.647558,0.030255
250304,0.122860,0.041348,0.473663,0.079800,0.739004,0.811371,0.276833,0.193411,0.731668,0.143221,0.136145,0.979055,0.180128,3.811077,0.827532,0.044156
250305,0.108818,0.040880,0.472455,0.094218,0.607553,0.860911,0.177458,0.186251,0.614708,0.164972,0.159992,0.974684,0.175060,3.749616,0.796911,0.042471


### Load 510 Dashboards data

In [5]:
risk = pd.read_csv('./data/510_community_risk.csv', sep=';')
risk_name = np.array(list(risk.columns))
impact = pd.read_csv('./data/510_impact_database.csv', sep=';')
impact_name = np.array(list(impact.columns))

### Flip signs of the indicators
Swap signs of the attributes expected to have a "negative" affect on vulnerability.


In [6]:
for name, sign, _, _ in input_names:
    if sign == 'neg':
        data[name] = -data[name].values
    elif sign == 'pos':
        pass
    else:
        raise Exception("problem")

In [10]:
data.head()

,PAGE5,PAGE65,PFEMALE,PDISABL,PMEDINS,PNOSWALL,PNOWATER,PNOELEC,PNOSEWAGE,PLITERACY,PNOPRIEDU,PNOCOLLEGE,PRENT,ANUMBFAMILY,PPHONE,PAUTOMOBILE
IDDIST,,,,,,,,,,,,,,,,
10101,0.080794,0.081132,0.526650,0.099573,-0.818497,0.049218,0.096125,0.075867,0.146295,0.089122,0.090089,0.848434,0.278042,3.652009,-0.926905,-0.106472
10102,0.083969,0.179389,0.488550,0.251908,-0.954198,0.720000,0.010000,0.150000,0.220000,0.184000,0.156000,0.960000,0.190000,2.466667,-0.625000,-0.000000
10103,0.097711,0.083627,0.475352,0.090669,-0.817782,0.585586,0.330330,0.276276,0.666667,0.178236,0.169794,0.981238,0.201201,3.000000,-0.695775,-0.036620
10104,0.068536,0.115265,0.506231,0.182243,-0.889408,0.278302,0.122642,0.122642,0.169811,0.099673,0.099673,0.968954,0.132075,2.918182,-0.740909,-0.022727
10105,0.083761,0.140171,0.482051,0.073504,-0.947009,0.446078,0.200980,0.343137,0.583333,0.186715,0.179533,0.987433,0.102941,2.839806,-0.567961,-0.000000


### Missing data control
INEI data and 510 data has missing data at some districts. Here, we fill those values by taking the average value of their neighbors. This is done using the spatial weights matrix `w`.

In [7]:
assert np.isnan(data).sum().sum() == 0

In [8]:
# import pysal as ps
# w = ps.lib.weights.Queen.from_shapefile('./data/DISTRITOS.shp', idVariable='IDDIST')
# w.transform = 'R'
# data.loc[np.isnan(data['PAGE5'])]
# ps.lib.weights.spatial_lag.lag_spatial(w, shp_fips.MHSEVAL_ALT)
# dbf = ps.lib.io.open('./data/DISTRITOS.dbf')
# iddist = dbf.by_col('IDDIST')
# shp_fips = pd.DataFrame(dbf.by_col('IDDIST'), index=iddist)
# shp_fips
# print(w.n)
# print(data.shape)

### Save the dataframe of input data

In [9]:
if True:
    fn = './data/inputdata_peru.hdf'
    data.to_hdf(fn, 'data')
    print('%s is saved.' % fn)
    fn = './data/inputdata_peru_table.hdf'
    data_name.to_hdf(fn, 'name')
    print('%s is saved.' % fn)

./data/inputdata_peru.hdf is saved.
./data/inputdata_peru_table.hdf is saved.


In [ ]:
# fhv.censusToRaster('./census/page5.tif', meta, did, page5)
# fhv.censusToRaster('./census/page5.tif', meta, did, page65)

In [ ]:
# Load a raster of district IDs
with rasterio.open(os.path.join('data', 'distid_30s.tif')) as src:
    did = src.read().squeeze()
    meta = src.meta.copy()
did.shape
meta